# Crowd data analysis:
## Crowd Anomaly behaviour 
### -Vishakha Bhat 
### 1DA17EC155
----------------------------------------------------------------------

# Upload all .py files 
# First create two folders training_vid and testing_vid 
# upload respective videos in respective folders from the Avenue dataset 
# Give the path for training_vid and create training.npy file by running the first block of code-processor.py -this gives you frames of the vids
------------------------------------------------------------ 


# Now train the model for your network by loading training.npy file and save the model as AnomalyDetector.h5 in your second block of code - train.py


---




# Now run processor.py for your testing_vid by giving the path testing_vid and create testing.npy file by running the third block of code

---



# Load AnomalyDetector.h5 file and testing.npy file to test the videos for anomaly
# in the fourth block of code viz basically test.py 
---







# Next we're trying this out for a single video 
# So create a folder called "randomvid" and into it upload the video and name it testvid.avi
# Rename the already formed trainig.npy to test.npy
# Run the fifth block and the following happens:
## "Give the source path as the randomvid folder and create a randomvid.npy and load AnomalyDetector.h5 file along with loading randomvid.npy file which does the detection of anomaly. further, since we also want the frame at which it happens, load the AnomalyDetector.h5 file and give the path of the VIDEO(not the folder)" 


---



In [1]:
from PIL import Image
#process.py for training videos-- extracting frames and stuff:-

from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
imagestore=[]

'''
parser=argparse.ArgumentParser(description='Source Video path')
parser.add_argument('Source Video path',type=str)
parser.add_argument('fps',type=int)
args=parser.parse_args()
'''
video_source_path='/content/training_vid'
fps=5
#try running the fps at 5. It is like that in the sample
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 
  #img=numpy.array(img.fromarray(arr).resize()).
  #img=resize(img,(227,227,3))
	#img=imresize(img,(227,227,3))
  
  #img = cv2.imread(img)
  #res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)

	#Convert the Image to Grayscale
	img=resize(img,(227,227,3))

	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)


#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," training video")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)

imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('training.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))



Using TensorFlow backend.


Found  16  training video


0

In [2]:
#model.py---training model:-

from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential



def load_model():
	"""
	Return the model used for abnormal event 
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model



#train.py--:-


from keras.callbacks import ModelCheckpoint, EarlyStopping
from model import load_model
import numpy as np 
import argparse
#from tensorflow.python import keras.models.Sequential
#from tensorflow.python import keras.layers.Dense
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Sequential

'''
parser=argparse.ArgumentParser()
parser.add_argument('n_epochs',type=int)
'''
#args=100

X_train=np.load('training.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()



epochs=100
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error", save_best_only=True)

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Model has been loaded')

	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )


Model has been loaded
Epoch 1/100
18/18 [==============================] - 40s 2s/step - loss: 0.2387 - accuracy: 0.5193
Epoch 2/100


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with mean_squared_error available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


18/18 [==============================] - 37s 2s/step - loss: 0.2077 - accuracy: 0.5469
Epoch 3/100
18/18 [==============================] - 37s 2s/step - loss: 0.2036 - accuracy: 0.5471
Epoch 4/100
18/18 [==============================] - 37s 2s/step - loss: 0.2007 - accuracy: 0.5469
Epoch 5/100
18/18 [==============================] - 37s 2s/step - loss: 0.1988 - accuracy: 0.5471
Epoch 6/100
14/18 [======================>.......] - ETA: 8s - loss: 0.1695 - accuracy: 0.5816 Model has been loaded
Epoch 1/100
18/18 [==============================] - 40s 2s/step - loss: 0.2387 - accuracy: 0.5193
Epoch 2/100


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with mean_squared_error available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


18/18 [==============================] - 37s 2s/step - loss: 0.2077 - accuracy: 0.5469
Epoch 3/100
18/18 [==============================] - 37s 2s/step - loss: 0.2036 - accuracy: 0.5471
Epoch 4/100
18/18 [==============================] - 37s 2s/step - loss: 0.2007 - accuracy: 0.5469
Epoch 5/100
18/18 [==============================] - 37s 2s/step - loss: 0.1988 - accuracy: 0.5471
Epoch 6/100
18/18 [==============================] - 37s 2s/step - loss: 0.1629 - accuracy: 0.5980
Epoch 7/100
Epoch 7/100
18/18 [==============================] - 43s 2s/step - loss: 0.1246 - accuracy: 0.6688
Epoch 8/100
Epoch 8/100
18/18 [==============================] - 37s 2s/step - loss: 0.1055 - accuracy: 0.6955
Epoch 9/100
Epoch 9/100
18/18 [==============================] - 37s 2s/step - loss: 0.0965 - accuracy: 0.7068
Epoch 10/100
Epoch 10/100
18/18 [==============================] - 37s 2s/step - loss: 0.0928 - accuracy: 0.7117
Epoch 11/100
Epoch 11/100
18/18 [==============================] - 36s 

In [3]:
#process.py for testing videos-- extracting frames and stuff:-

from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
imagestore=[]

'''
parser=argparse.ArgumentParser(description='Source Video path')
parser.add_argument('Source Video path',type=str)
parser.add_argument('fps',type=int)
args=parser.parse_args()
'''
video_source_path='/content/testing_vid'
fps=5
#try running the fps at 5. It is like that in the sample
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 
  #img=numpy.array(img.fromarray(arr).resize()).
  #img=resize(img,(227,227,3))
	#img=imresize(img,(227,227,3))
  
  #img = cv2.imread(img)
  #res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)

	#Convert the Image to Grayscale
	img=resize(img,(227,227,3))

	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)


#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)

imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('testing.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))



Found  21  testing videos


0

In [4]:
'''

Testing module to test the presence of Anomalous Events in a Video

The module computes reconstruction loss between input bunch and

the reconstructed batch from the model, and flagges the batch as anomalous
if loss value is greater than a given threshold.








Author: Harsh Tiku

'''


from keras.models import load_model
import numpy as np 




def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''




	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist



'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004244678


model=load_model('AnomalyDetector.h5')

X_test=np.load('testing.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)

for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
 

	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		flag=1


	else:
		print('Bunch Normal')



if flag==1:
	print("Anomalous Events detected")






Anomalous bunch of frames at bunch number 0
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Anomalous bunch of frames at bunch number 5
Anomalous bunch of frames at bunch number 6
Anomalous bunch of frames at bunch number 7
Anomalous bunch of frames at bunch number 8
Anomalous bunch of frames at bunch number 9
Anomalous bunch of frames at bunch number 10
Bunch Normal
Bunch Normal
Bunch Normal
Bunch Normal
Anomalous bunch of frames at bunch number 15
Bunch Normal
Anomalous bunch of frames at bunch number 17
Anomalous bunch of frames at bunch number 18
Anomalous bunch of frames at bunch number 19
Anomalous bunch of frames at bunch number 20
Anomalous bunch of frames at bunch number 21
Anomalous bunch of frames at bunch number 22
Bunch Normal
Bunch Normal
Anomalous Events detected


In [56]:


from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
imagestore=[]

'''
parser=argparse.ArgumentParser(description='Source Video path')
parser.add_argument('Source Video path',type=str)
parser.add_argument('fps',type=int)
args=parser.parse_args()
'''
video_source_path='/content/randomvid'
fps=5

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 
  #img=numpy.array(img.fromarray(arr).resize()).
  #img=resize(img,(227,227,3))
	#img=imresize(img,(227,227,3))
  
  #img = cv2.imread(img)
  #res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)

	#Convert the Image to Grayscale
	img=resize(img,(227,227,3))

	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)


#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," video to test")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)

imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('randomvid.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))



from keras.models import load_model
import numpy as np 




def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''




	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist



'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

threshold=0.0004244678


model=load_model('AnomalyDetector.h5')

X_test=np.load('randomvid.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)

for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
 

	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		flag=1


	else:
		print("Bunch Normal at bunch number {}".format(number))



if flag==1:
	print("Anomalous Events detected")


import cv2
from model import load_model
import numpy as np 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from test import mean_squared_loss
from keras.models import load_model
import argparse
import sys
import skimage.viewer




modelpath='AnomalyDetector.h5'


vc=cv2.VideoCapture('/content/randomvid/testvid.avi') #SET VID LOC
rval=True
print('Loading model')
model=load_model(modelpath)
print('Model loaded')

#threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
threshold=0.0004583678 #(Accuracy level Vishakha)
counter=0
while (vc.isOpened()):
	imagedump=[]
	for i in range(20):
		rval,frame=vc.read()
		try:
			frame=resize(frame,(227,227,3))
		except:
			print("\n--------------Video ended--------------\n");exit();quit();sys.exit()
		#Convert the Image to Grayscale
		gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
		gray=(gray-gray.mean())/gray.std()

		gray=np.clip(gray,0,1)
		imagedump.append(gray)


	imagedump=np.array(imagedump)

	imagedump.resize(227,227,10)
	imagedump=np.expand_dims(imagedump,axis=0)
	imagedump=np.expand_dims(imagedump,axis=4)

	counter=counter+1
	print('Processing data for frame :',counter)

	output=model.predict(imagedump)



	loss=mean_squared_loss(imagedump,output)
 
 

	if loss>threshold:
   

		print('Anomalies Detected in frame ',counter)
		#viewer=skimage.viewer.ImageViewer(imagedump)
		#viewer.show()
  	
    
    
    
  


	else :
		print('No anomalies detected in frame',counter)
print('-------------------------------------------')

print("---------------------------------\nVideo complete\n")





Found  1  video to test
Anomalous bunch of frames at bunch number 0
Anomalous bunch of frames at bunch number 1
Anomalous Events detected
Loading model
Model loaded
Processing data for frame : 1
Anomalies Detected in frame  1
Processing data for frame : 2
Anomalies Detected in frame  2
Processing data for frame : 3
Anomalies Detected in frame  3
Processing data for frame : 4
Anomalies Detected in frame  4
Processing data for frame : 5
Anomalies Detected in frame  5
Processing data for frame : 6
Anomalies Detected in frame  6
Processing data for frame : 7
Anomalies Detected in frame  7
Processing data for frame : 8
Anomalies Detected in frame  8
Processing data for frame : 9
Anomalies Detected in frame  9
Processing data for frame : 10
Anomalies Detected in frame  10
Processing data for frame : 11
Anomalies Detected in frame  11
Processing data for frame : 12
Anomalies Detected in frame  12
Processing data for frame : 13
Anomalies Detected in frame  13
Processing data for frame : 14
Ano

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
